In [1]:
from urllib.request import urlopen
from urllib.parse import quote_plus as qp #문자 아스키코드로 변환
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
import os

In [2]:
#인터넷 창 띄우기(chrome driver에 각자 개별 로그인)
driver = webdriver.Chrome()
url = 'https://www.instagram.com/'
driver.get(url)

In [3]:
#검색어 태그로 인터넷 창 이동
# 홈페이지 업데이트 했는지 class 명이 바뀜
class_name = '_aagt'
search = '이동간받이' #검색어
baseurl = 'https://www.instagram.com/explore/tags/'
url = baseurl + qp(search)
driver.get(url)
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME,class_name)))
imgurl=[] #이미지 url 저장 list

In [4]:
scroll_num = 100 #스크롤 횟수
scroll_pause_time=1.5 #스크롤 내릴때 이미지 로딩되는 시간 기다림
upscroll=-100
pic_num = 800 # 최대로 가져올 사진의 수

#스크롤 하기
#스크롤은 기다리기 귀찮으면 직접 한 후에 아래 코드 실행해도 됨
#인터넷 환경에 따라 이미지 로딩이 느릴때, 스크롤이 적게 실행될 수 있음
breaker = [] # 업데이트 5회이상 안될시, break


for i in range(scroll_num+1):

    #파싱
    html = driver.page_source
    soup = BeautifulSoup(html)
    insta = soup.select('.'+class_name)
    for pic in insta:
        imgurl.append(pic['src']) #이미지 주소 가져오기
        imgurl = (list(set(imgurl))) #이미지주소 중복 제거
    
    print('스크롤 {}번 실행, {}개 그림을 가져올 수 있습니다.'.format(i, len(list(set(imgurl)))))
    
    breaker.append(len(list(set(imgurl))))
    time.sleep(scroll_pause_time)
    last_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)
    new_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script('scrollBy(0,{})'.format(upscroll))
    
    if new_height == last_height:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        driver.execute_script('scrollBy(0,{})'.format(upscroll))
        if new_height == last_height:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(scroll_pause_time)
            new_height = driver.execute_script("return document.body.scrollHeight")
            driver.execute_script('scrollBy(0,{})'.format(upscroll))
        else:
            last_height = new_height
            continue
    if len(breaker)>5 and breaker[-5]==breaker[-1]: 
        print('더이상 update가 없습니다.')
        break
    if breaker[-1] >= pic_num:
        print(f'사진의 수가 {breaker[-1]}개로 최대개수인 {pic_num}개 이상입니다')
        break

스크롤 0번 실행, 33개 그림을 가져올 수 있습니다.
스크롤 1번 실행, 39개 그림을 가져올 수 있습니다.
스크롤 2번 실행, 51개 그림을 가져올 수 있습니다.
스크롤 3번 실행, 57개 그림을 가져올 수 있습니다.
스크롤 4번 실행, 69개 그림을 가져올 수 있습니다.
스크롤 5번 실행, 81개 그림을 가져올 수 있습니다.
스크롤 6번 실행, 87개 그림을 가져올 수 있습니다.
스크롤 7번 실행, 103개 그림을 가져올 수 있습니다.
스크롤 8번 실행, 121개 그림을 가져올 수 있습니다.
스크롤 9번 실행, 130개 그림을 가져올 수 있습니다.
스크롤 10번 실행, 149개 그림을 가져올 수 있습니다.
스크롤 11번 실행, 169개 그림을 가져올 수 있습니다.
스크롤 12번 실행, 196개 그림을 가져올 수 있습니다.
스크롤 13번 실행, 218개 그림을 가져올 수 있습니다.
스크롤 14번 실행, 227개 그림을 가져올 수 있습니다.
스크롤 15번 실행, 249개 그림을 가져올 수 있습니다.
스크롤 16번 실행, 258개 그림을 가져올 수 있습니다.
스크롤 17번 실행, 288개 그림을 가져올 수 있습니다.
스크롤 18번 실행, 309개 그림을 가져올 수 있습니다.
스크롤 19번 실행, 318개 그림을 가져올 수 있습니다.
스크롤 20번 실행, 339개 그림을 가져올 수 있습니다.
스크롤 21번 실행, 348개 그림을 가져올 수 있습니다.
스크롤 22번 실행, 369개 그림을 가져올 수 있습니다.
스크롤 23번 실행, 375개 그림을 가져올 수 있습니다.
스크롤 24번 실행, 396개 그림을 가져올 수 있습니다.


KeyboardInterrupt: 

In [5]:
num = len(os.listdir('original_pictures')) #class 갯수
os.mkdir('./original_pictures/'+'{}_'.format(num+1)+search)  #폴더 생성
for i, pic in enumerate(imgurl):
    with urlopen(pic) as f: #날짜 시간 포함하여 이미지 저장
        with open('./original_pictures/'+'{}_'.format(num+1)+search+'/'+search+'_'+'{0:04d}'.format(i+1)+'_'+time.strftime('%y%m%d-%H%M')+'.jpg','wb') as h:
            img = f.read()
            h.write(img)
    print('이미지 저장 중 : {}/{}'.format(i+1,len(imgurl)), end='\r', flush=True)